# A trivial SOS decomposition example

**Contributed by**: votroto
**Adapted from**: Examples 3.25 of [BPT12]

[BPT12] Blekherman, G.; Parrilo, P. A. & Thomas, R. R.
*Semidefinite Optimization and Convex Algebraic Geometry*.
Society for Industrial and Applied Mathematics, **2012**.

In [1]:
using DynamicPolynomials
using SumOfSquares
import CSDP

The polynomial `p = x^2 - x*y^2 + y^4 + 1` is SOS.
We can, for example, decompose it as
`p = 3/4*(x - y^2)^2 + 1/4*(x + y)^2 + 1`,
which clearly proves that `p` is SOS, and there are infinitely many other ways
to decompose `p` into sums of squares.

We can use SumOfSquares.jl to find such decompositions.

First, setup the polynomial of interest.

In [2]:
@polyvar x y
p = x^2 - x*y^2 + y^4 + 1

y⁴ - xy² + x² + 1

Secondly, constrain the polynomial to be nonnegative.
SumOfSquares.jl transparently reinterprets polyonmial nonnegativity as the
appropriate SOS certificate for polynomials nonnegative on semialgebraic sets.

In [3]:
model = SOSModel(CSDP.Optimizer)
@constraint(model, cref, p >= 0)

cref : (1)y⁴ + (-1)xy² + (1)x² + (1) is SOS

Thirdly, optimize the feasibility problem!

In [4]:
optimize!(model)

CSDP 6.2.0
This is a pure primal feasibility problem.
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.00e-01 Pobj:  0.0000000e+00 Ad: 1.00e+00 Dobj:  2.7659722e+01 
Iter:  2 Ap: 1.00e+00 Pobj:  0.0000000e+00 Ad: 1.00e+00 Dobj:  1.2562719e+01 


Lastly, recover a SOS decomposition.
In general, SOS decompositions are not unique!

In [5]:
sos_dec = sos_decomposition(cref, 1e-4)

(-0.9492757372229984*y^2 + 0.591634214595468*x + 0.7423566403018638)^2 + (-1.1201589623631873*y)^2 + (3.952421508053554e-16*y^2 + 0.7820242435285941*x - 0.6232480104529253)^2 + (-0.314444867536*y^2 - 0.19597713808894554*x - 0.24590350966628946)^2

Converting, rounding, and simplifying - Huzza, Back where we began!

In [6]:
polynomial(sos_dec, Float32)

y⁴ - xy² + x² - 2.7755576e-16y² + 1.6653345e-16x + 1.0

## A deeper explanation and the unexplained `1e-4` parameter

`p = x^2 - x*y^2 + y^4 + 1` can be represented in terms of its Gram matrix as

In [7]:
gram = gram_matrix(cref)

GramMatrix{Float64, MonomialBasis{DynamicPolynomials.Monomial{true}, DynamicPolynomials.MonomialVector{true}}, Float64, SymMatrix{Float64}}([1.0 -0.49999999999999994 0.0 -0.6273780504812863; -0.49999999999999994 1.0 0.0 3.02535774210355e-17; 0.0 0.0 1.2547561009625725 0.0; -0.6273780504812863 3.02535774210355e-17 0.0 1.0], MonomialBasis{DynamicPolynomials.Monomial{true}, DynamicPolynomials.MonomialVector{true}}(DynamicPolynomials.Monomial{true}[y², x, y, 1]))

In [8]:
gram.basis.monomials' * gram.Q * gram.basis.monomials

y⁴ - 0.9999999999999999xy² + x² + 6.0507154842071e-17x + 1.0

where the matrix `gram.Q` is positive semidefinite, because `p` is SOS. If we
could only get the decomposition `gram.Q = V' * V`, the SOS decomposition would
simply be `||V * monomials||^2`.

Unfortunately, we can not use Cholesky decomposition, since gram `Q` is only
semidefinite, not definite. Hence, SumOfSquares.jl uses SVD decomposition
instead and discards small singular values (in our case `1e-4`).

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*